# Explore new dataset from Gießen

Preprocessing. 

get some descriptive stats on the sample we have here. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pdb

pd.set_option('display.max_columns', None)

cur_dir = os.getcwd()
path_to_data = os.path.join(cur_dir, "data_trustgame_depression")
os.chdir(path_to_data)

In [2]:
# negative to positive data
df_np = pd.read_csv('trustgame_neg_to_pos.csv')
df_pn = pd.read_csv('trustgame_pos_to_neg.csv')
df_a  = pd.read_csv('trustgame_ambiguous.csv')

In [3]:
# start cleaning this data. Start with df_np bc we use this for DPK a bit. 
# drop unneeded cols
unwanted_cols = ['Event Index', 'UTC Timestamp', 'UTC Date and Time', 'Local Timestamp',
       'Local Timezone', 'Local Date and Time', 'Experiment ID','Participant Public ID',
       'Experiment Version', 'Tree Node Key', 'Repeat Key', 'Schedule ID','practice trial keep', 'practice trial share',
       'Participant External Session ID', 'Participant Completion Code', 'Reaction Onset','Response Type',
       'Participant Device Type', 'Participant Device', 'Participant OS','Dishonest',
       'Participant Browser', 'Participant Monitor Size', 'Participant Starting Group',
       'Participant Viewport Size', 'Checkpoint', 'Room ID', 'Room Order','Zone Name',
       'checkpoint-jpdv', 'randomiser-i3f2', 'X Coordinate', 'Y Coordinate','randomise_blocks','randomise_trials',
       'checkpoint-neje', 'order-73e3', 'checkpoint-iq13', 'checkpoint-t4s8','Timed Out', 'ANSWER']

df_np = df_np.drop(unwanted_cols, axis=1)
df_pn = df_pn.drop(unwanted_cols, axis=1)
df_a  = df_a.drop(unwanted_cols, axis=1)

In [4]:
# now we start filtering. 
df_np = df_np[df_np['display']=='Trials']
df_pn = df_pn[df_pn['display']=='Trials']
df_a  = df_a[df_a['display']=='Trials']
print(df_np.shape, df_pn.shape, df_a.shape)
df_np = df_np[df_np['Zone Type']=='response_button_text']
df_pn = df_pn[df_pn['Zone Type'] == 'response_button_text']
df_a  = df_a[df_a['Zone Type'] == 'response_button_text']
print(df_np.shape, df_pn.shape, df_a.shape)

(1542, 26) (2229, 26) (1797, 26)
(420, 26) (600, 26) (480, 26)


In [5]:
vp_pn = list(df_pn['Participant Private ID'].unique())
vp_np = list(df_np['Participant Private ID'].unique())
vp_a  = list(df_a['Participant Private ID'].unique())
print(f'N pos-neg: {len(vp_pn)}')
print(f'N neg-pos: {len(vp_np)}')
print(f'ambiguous: {len(vp_a)}')

N pos-neg: 10
N neg-pos: 7
ambiguous: 8


In [6]:
# merge all of them
df_list = [df_np, df_pn, df_a]
df = pd.concat(df_list)

In [7]:
# now we replace values so we can work with it. 
df['Response'] = df['Response'].replace({'50ct spenden': 1, '50ct behalten': 0}) # now 1 = spenden, 0 = behalten
df['partnerAnswer'] = df['partnerAnswer'].replace({'Entscheidung von Mitspieler(in): "Nichts zurückgeben"': 0, 
                                                  'Entscheidung von Mitspieler(in): "1€ zurückgeben"': 1}) # 0: abuse, 1: coop

# list of all subjects
all_vp = list(df['Participant Private ID'].unique())
print(f'All VP: {len(all_vp)}')

All VP: 25


In [8]:
# write into csv file for later usage
df.to_csv('data_n25_inclAmbiguous.csv')

In [14]:
# preprocess the BDI survey data.
df_bdi = pd.read_csv('data_bdi.csv')

survey_unwanted_cols = ['Event Index', 'UTC Timestamp', 'UTC Date and Time', 'Local Timestamp',
       'Local Timezone', 'Local Date and Time', 'Experiment ID',
       'Experiment Version', 'Tree Node Key', 'Repeat Key', 'Schedule ID',
       'Participant Public ID', 'Participant Starting Group', 'Participant Status',
       'Participant Completion Code', 'Participant External Session ID',
       'Participant Device Type', 'Participant Device', 'Participant OS',
       'Participant Browser', 'Participant Monitor Size',
       'Participant Viewport Size', 'Checkpoint', 'Room ID', 'Room Order','checkpoint-jpdv', 'randomiser-i3f2',
       'checkpoint-neje', 'order-73e3', 'checkpoint-iq13', 'checkpoint-t4s8',
       'Randomise questionnaire elements?']

df_bdi = df_bdi.drop(survey_unwanted_cols, axis=1)

In [15]:
# items
bdi_items = ['BDI_' + f'0{i}' for i in range(1,10)] + ['BDI_' + f'{i}' for i in range(10, 22)]
# responses
bdi_resp = [i + '-quantised' for i in bdi_items]
# all participants
vp_bdi = list(df_bdi['Participant Private ID'].unique())

# preprocess survey data to make it easier to use
# Drop rows with NaN values in the "Question Key" column
df_bdi = df_bdi.dropna(subset=["Question Key"])

# Filter out rows that are not relevant
df_bdi = df_bdi[df_bdi["Question Key"].str.startswith("BDI")]
df_items = df_bdi[~df_bdi["Question Key"].str.contains("quantised")]
df_quantised = df_bdi[df_bdi["Question Key"].str.contains("quantised")]

In [16]:
df_items["Question Number"] = df_items["Question Key"].str.replace("BDI_", "").astype(int)
df_quantised["Question Number"] = df_quantised["Question Key"].str.replace("BDI_", "").str.replace("-quantised", "").astype(int)

C:\Users\annae\AppData\Local\Temp\ipykernel_3696\4061585974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items["Question Number"] = df_items["Question Key"].str.replace("BDI_", "").astype(int)
C:\Users\annae\AppData\Local\Temp\ipykernel_3696\4061585974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quantised["Question Number"] = df_quantised["Question Key"].str.replace("BDI_", "").str.replace("-quantised", "").astype(int)


In [17]:
# restructure the whole dataframe
df_merged = pd.merge(df_items, df_quantised, on=["Participant Private ID", "Task Name", "Task Version", "Question Number"], suffixes=('_item', '_quantised'))
final_df = df_merged[["Participant Private ID", "Task Name", "Task Version", "Question Key_item", "Response_item", "Response_quantised"]]
final_df.columns = ["Participant Private ID", "Task Name", "Task Version", "Question Key", "Item", "Response"]
df_bdi = final_df
df_bdi

,Participant Private ID,Task Name,Task Version,Question Key,Item,Response
0,9253510.0,BDI-II,6.0,BDI_01,1 Ich bin oft traurig,2
1,9253510.0,BDI-II,6.0,BDI_02,2 Ich bin mutlos und erwarte nicht dass meine ...,3
2,9253510.0,BDI-II,6.0,BDI_03,1 Ich habe häufiger Versagensgefühle,2
3,9253510.0,BDI-II,6.0,BDI_04,1 Ich kann die Dinge nicht mehr so genießen wi...,2
4,9253510.0,BDI-II,6.0,BDI_05,1 Ich habe oft Schuldgefühle wegen Dingen die ...,2
...,...,...,...,...,...,...
520,8329948.0,BDI-II,6.0,BDI_17,1 Ich bin reizbarer als sonst,2
521,8329948.0,BDI-II,6.0,BDI_18,0 Mein Appetit hat sich nicht verändert,1
522,8329948.0,BDI-II,6.0,BDI_19,1 Ich kann mich nicht mehr so gut konzentriere...,2
523,8329948.0,BDI-II,6.0,BDI_20,1 Ich werde schneller müder oder erschöpft als...,2


In [18]:
print('BDI data for N = ',len(vp_bdi))
vp_bdi.pop(-1) # remove nan value

BDI data for N =  26


nan

In [19]:
bdi_res = dict()
for vp in vp_bdi: 
    df_sbj = df_bdi[df_bdi['Participant Private ID'] == vp]
    vp_score = df_sbj['Response'].astype(int).sum() -21 # minus 21 bc I think it is scored 0-4
    bdi_res[vp] = vp_score
    
bdi_median = np.median(list(bdi_res.values()))
print('median value BDI: ', bdi_median)

median value BDI:  22.0


In [20]:
bdi_res

{9253510.0: 27,
 9164610.0: 30,
 9130100.0: 11,
 9118973.0: 7,
 9114231.0: 17,
 9113722.0: 40,
 9089012.0: 10,
 9066992.0: 44,
 9054278.0: 17,
 9053831.0: 35,
 8964265.0: 32,
 8879711.0: 22,
 8843379.0: 20,
 8842342.0: 9,
 8816980.0: 15,
 8801433.0: 20,
 8796180.0: 35,
 8742634.0: 50,
 8714599.0: 18,
 8616319.0: 19,
 8552546.0: 35,
 8502516.0: 30,
 8478866.0: 22,
 8459387.0: 27,
 8329948.0: 15}

In [21]:
bdi = pd.DataFrame(list(bdi_res.items()), columns=['sbj_id', 'bdi_score'])
bdi.to_csv('bdi_n17.csv')

In [22]:
# add BDI value to the dataframe. 
df['BDI'] = df['Participant Private ID'].map(bdi_res)
df.to_csv('data_inclBDI_n25.csv')

In [23]:
df_a = df[df['Spreadsheet Name']=='Ambigious']
df_a.to_csv('data_ambiguous_only.csv')